<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/agents_and_routing/examples_react_agent_from_the_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup
Install required libraries and start a LLM using OLLAMA in the background

In [1]:
!pip install ollama langchain_community --quiet

modelid="gemma3:12b"

get_ipython().system_raw("curl -fsSL https://ollama.com/install.sh | sh")
get_ipython().system_raw("ollama serve &")
get_ipython().system_raw(f"ollama pull {modelid}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00


In [2]:

get_ipython().system_raw(f"ollama pull {modelid}")

In [4]:
!pip install ollama-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.6/46.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ollama 0.4.7 requires httpx<0.29,>=0.27, but you have httpx 0.26.0 which is incompatible.
google-genai 1.4.0 requires httpx<1.0.0dev,>=0.28.1, but you have httpx 0.26.0 which is incompatible.


In [3]:
import rich
import logging
from rich.logging import RichHandler
from rich import print
from ollama import chat
from ollama import ChatResponse
from ollama import Client
import re

# model="llama3.2"
model="gemma3:12b"
host="http://localhost:11434"

FORMAT = "%(message)s"
logging.basicConfig(
    level=logging.INFO, format=FORMAT, datefmt="[%X]", handlers=[RichHandler()]
)

log = logging.getLogger("simpleagent")
log.setLevel(logging.DEBUG)

system_prompt = """
Think step by step. You run in a loop of THOUGHT, ACTION, PAUSE, OBSERVATION.
At the end of the loop you output an ANSWER
Use THOUGHT to describe your thoughts about the question you have been asked.
Use ACTION to run one of the actions available to you - then return PAUSE.
OBSERVATION will be the result of running those actions.

Your available actions are:
### ACTIONS


Example session:

Question: What is the mass of Earth times 2?
THOUGHT: I need to find the mass of Earth
ACTION: get_planet_mass: Earth
PAUSE

You will be called again with this:

OBSERVATION: 5.972e24

THOUGHT: I need to multiply this by 2
ACTION: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

OBSERVATION: 1,1944×10e25

If you have the answer, output it as the ANSWER.

ANSWER: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:

""".strip()

def calculate(operation:str) -> float:
    """
    e.g. calculate: 4 * 7 / 3
    Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary
    """
    log.info(f"calculate: {operation}")
    return eval(operation)

def get_planet_mass(planet:str)->float:
    """
    e.g. get_planet_mass: Earth
    returns weight of the planet in kg
    """
    log.info(f"return mass of {planet} ")
    return 3.285e23

class BaseAgent:
    """
    The Base Agent class.
    This class holds a memory of previous calls and builds the foundation for agents
    """
    def __init__(self, systemprompt: str="", tools=[]):
        self.log=logging.getLogger("agent")
        self.systemprompt=systemprompt

        toolprompt=""
        for tool in tools:
            toolprompt=toolprompt+tool.__name__+":"+tool.__doc__+"\n\n"
        self.systemprompt=self.systemprompt.replace("### ACTIONS","### ACTIONS\n\n"+toolprompt)

        log.debug(self.systemprompt)
        self.clear_memory()
        self.client = Client(
         #   host='http://192.168.0.9:11434',
            host=host,
            headers={'x-some-header': 'some-value'}
        )

    def __add2Memory(self, role: str, message: str=""):
        if message:
            self.memory.append({ 'role':role, 'content': message})

    def __execute(self) -> str:
        self.log.debug(f'calling with history : {self.memory}')
        response: ChatResponse = self.client.chat(model=model, messages=self.memory,options={'temperature':0})
        self.log.info(response)
        self.__add2Memory(response.message.role, response.message.content)
        return response.message.content

    def __call__(self, prompt:str=""):
        if prompt:
            self.__add2Memory("user",prompt)
        response= self.__execute()
        return response

    def clear_memory(self):
        self.memory=[]
        if self.systemprompt:
            self.__add2Memory("system", self.systemprompt)

class ReActAgent(BaseAgent):
    def __init__(self, systemprompt: str="", tools=[]):
        super().__init__(systemprompt,tools)
        self.max_iterations=10
    def __call__(self, prompt:str=""):
        loopcount=0
        message=prompt
        while loopcount<self.max_iterations:
            response=super().__call__(message)
            if "PAUSE" in response and "ACTION" in response:
                action = re.findall(r"ACTION: ([a-z_]+): (.+)", response, re.IGNORECASE)
                tool = action[0][0]
                arg = action[0][1]
                result=eval(f"{tool}('{arg}')")
                message=f"OBSERVATION: {result}"
            if "ANSWER" in response:
                break
            loopcount+=1

        return response

if __name__=='__main__':
    log.info ("----------------\n\nSimple agent\n\n-----------------\n")
    log.info("launching!")

    agent=ReActAgent(systemprompt=system_prompt,tools=[get_planet_mass,calculate])
    response = agent('What is the mass of Mercury times 5?')
    print (response)







INFO:simpleagent:----------------

Simple agent

-----------------

INFO:simpleagent:launching!
DEBUG:simpleagent:Think step by step. You run in a loop of THOUGHT, ACTION, PAUSE, OBSERVATION.
At the end of the loop you output an ANSWER
Use THOUGHT to describe your thoughts about the question you have been asked.
Use ACTION to run one of the actions available to you - then return PAUSE.
OBSERVATION will be the result of running those actions.

Your available actions are:
### ACTIONS

get_planet_mass:
    e.g. get_planet_mass: Earth
    returns weight of the planet in kg
    

calculate:
    e.g. calculate: 4 * 7 / 3
    Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary
    




Example session:

Question: What is the mass of Earth times 2?
THOUGHT: I need to find the mass of Earth
ACTION: get_planet_mass: Earth
PAUSE 

You will be called again with this:

OBSERVATION: 5.972e24

THOUGHT: I need to multiply this by 2
ACTION: calcu

THOUGHT: I have calculated the mass of Mercury times 5. I can now output the answer.
ANSWER: The mass of Mercury times 5 is 1.6425e+24.